In [2]:
import numpy as np 
import pandas as pd


In [19]:
df1 = pd.read_csv('data/fraud.csv')
df2 = pd.read_csv('data/debit.csv')


In [21]:
filtered_df = df[(df['OLD_DISPONIBLE'] > 0) & (df['DISPONIBLE'] == 0)]
filtered_df.head()

KeyError: 'OLD_DISPONIBLE'

In [3]:
df1 = pd.read_csv('data/ml_debit.csv')
df2 = pd.read_csv('data/ml_credit.csv', nrows=7000)

In [4]:
df = pd.concat([df1, df2], ignore_index=True)
df.shape

(13229, 9)

In [5]:
df = df.sample(frac=1).reset_index(drop=True)
df["target"] = df["target"].map({"debit":1,"credit":0})

In [6]:
columns_to_convert = ['sum_amount', 'mean_amount', 'min_amount', 'max_amount', 'num_debits', 'num_credits']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')
df.dtypes

account         object
step            object
sum_amount     float64
mean_amount    float64
min_amount     float64
max_amount     float64
num_debits     float64
num_credits    float64
target         float64
dtype: object

In [7]:
df = df.dropna()
df.isnull().sum()


account        0
step           0
sum_amount     0
mean_amount    0
min_amount     0
max_amount     0
num_debits     0
num_credits    0
target         0
dtype: int64

In [8]:
df.describe()

,sum_amount,mean_amount,min_amount,max_amount,num_debits,num_credits,target
count,1.188000e+04,1.188000e+04,1.188000e+04,1.188000e+04,11880.000000,11880.000000,11880.000000
mean,1.096729e+08,2.249041e+07,3.839886e+06,6.690132e+07,3.650253,0.919697,0.428704
std,2.165601e+09,4.346825e+08,1.293754e+08,1.335662e+09,1.372152,1.046418,0.494912
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,1.639600e+05,3.801500e+04,5.000000e+03,7.799725e+04,3.000000,0.000000,0.000000
50%,5.000000e+05,1.120224e+05,1.640600e+04,2.000000e+05,4.000000,1.000000,0.000000
75%,1.708778e+06,3.927890e+05,1.000000e+05,1.000000e+06,5.000000,1.000000,1.000000
max,1.760013e+11,3.520027e+10,1.327980e+10,8.800000e+10,5.000000,5.000000,1.000000


In [9]:
df["target"].value_counts()

target
0.0    6787
1.0    5093
Name: count, dtype: int64

In [10]:
df["step"] = df["step"].apply(pd.to_datetime)


In [11]:
df.corr()['target']

account        0.712715
step          -0.191912
sum_amount     0.014106
mean_amount    0.014839
min_amount     0.007513
max_amount     0.009752
num_debits    -0.152937
num_credits    0.084848
target         1.000000
Name: target, dtype: float64

In [17]:
df['target'].value_counts()
df.drop

KeyError: "['account'] not found in axis"